### Scraping Comments https://www.blackwhitereadallover.com/

In [34]:
# Libraries

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from random import randint
from time import sleep 


In [35]:
# Setting up the Chrome webdriver

options = webdriver.ChromeOptions()
#options.add_argument('--headless') #uncomment to not render driven browser during scrapping

options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


#### Scraping the Archives

In [36]:
# Create a list of archive URLs (Feb, 2007 - March, 2020)

archive_periods = []

for j in range(2007,2021):
    for k in range(1,12):
        single_period = 'https://www.blackwhitereadallover.com/archives/'+str(j)+'/'+str(k)
        archive_periods.append(single_period)
        ;
        
# Subsetting out some periods for which there are no articles
active_archives = archive_periods[1:len(archive_periods)-8]

#len(active_archives) #145
        

In [37]:
## Code to check from 'Load More' button on each archive page and clicking it if found

#driver.get('https://www.blackwhitereadallover.com/archives/2019/8')
# https://www.blackwhitereadallover.com/archives/2012/7

# Checking to see if 'Load More' button exists (for months with>30 articles) and if so clicking it
#button = driver.find_elements_by_xpath("//button[contains(text(),'Load More')]")
#if len(button) > 0 :
#    sleep(3)
#    button[0].click()


In [48]:
## Collecting the article URLS from each archive page, storing in 'archive_links' list

archive_links = []

for u in range(0,len(archive_periods)):
    #print(u)
    
    sleep(randint(4, 7))  # sleep between 1 and 3 seconds
    
    driver.get(active_archives[u])
    
    
    # Checking to see if 'Load More' button exists (for months with>30 articles) and if so clicking it
    button = driver.find_elements_by_xpath("//button[contains(text(),'Load More')]")
    if len(button) > 0 :
        sleep(7)
        button[0].click()
    else:
        pass
 
    
    # Retrieving number of articles from title 
    # OLD archive_title = driver.find_element_by_xpath('/html/body/div[1]/div[2]/h1')
    archive_title = driver.find_element_by_xpath("//h1[contains(text(),'Archives for')]")

    #print(archive_title.text)
    
    start = archive_title.text.find('(')+1
    start = archive_title.text[start:]
    articles = int(start.replace(')',''))
    #print(articles)
    
    
    # Retrieving the article links themselves
    for q in range(0,articles):
        
        article_link = driver.find_elements_by_css_selector('h2.c-entry-box--compact__title>a')
        article_link = article_link[q].get_attribute('href')
        
    
        archive_links.append(article_link) 
           
    ;



IndexError: list index out of range

In [52]:
print(archive_links)

#len(archive_periods)

['https://www.blackwhitereadallover.com/2007/2/28/2836773/february-27-afternoon-quick-notes', 'https://www.blackwhitereadallover.com/2007/2/27/2836772/juventus-modena-wrap-up-leaving-it-late', 'https://www.blackwhitereadallover.com/2007/2/23/2836771/thursday-feb-22-quick-notes-crybabies-politics-cocaine-lindsay-lohan', 'https://www.blackwhitereadallover.com/2007/2/22/2836770/weekend-wrap-up-a-little-tardy', 'https://www.blackwhitereadallover.com/2007/2/16/2836769/stop-the-presses-tudor-stays-and-other-junk', 'https://www.blackwhitereadallover.com/2007/2/15/2836768/february-15-2007-quick-notes', 'https://www.blackwhitereadallover.com/2007/2/15/2836767/primavera-player-profile-sebastian-giovinco', 'https://www.blackwhitereadallover.com/2007/2/14/2836766/march-12-2003-igor-tudor-day', 'https://www.blackwhitereadallover.com/2007/3/30/2836784/juventus-pescara-preview', 'https://www.blackwhitereadallover.com/2007/3/21/2836783/3-21-07-juventus-news-from-drunks-to-phenoms-and-metzelder-co', 'h

#### Logging in to BRWAO

In [54]:
# Logging in to BWRAO to have access to comment recs

driver.get("https://auth.voxmedia.com/login?return_to=https://www.blackwhitereadallover.com/")

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "p-text-input"))
    )

username = driver.find_element_by_xpath('//*[@id="login[username]"]')
username.clear()
username.send_keys("rsolter")


password = driver.find_element_by_xpath('//*[@id="login[password]"]')
password.clear()
password.send_keys("pass123!")


button = driver.find_element_by_xpath('//*[@id="auth"]/div/form/fieldset[3]/input')
button.click()

In [55]:
# Have selenium wait until the BWRAO logo loads (therefore, has logged in properly)
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )

#### Scraping relevant info from individual pages

In [56]:
## Gathering blog title, author, publish date and comment list and comment count from a select page

### Insert loop here to visit relevant URLs

driver.get("https://www.blackwhitereadallover.com/2020/4/3/21206623/miralem-pjanic-marco-verratti-juventus-paris-saint-germain-2020-serie-a-summer-transfer-rumors#comments")

## Grab article title, author

blog_title = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[2]/h1')
print(blog_title.text)

article_author = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[1]/a/span')
print(article_author.text)

article_publish_date = driver.find_element_by_xpath('//*[@id="content"]/article/div[1]/div[1]/div[3]/span[1]/span[2]/time')
print(article_publish_date.text)

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list"))
    )

#Find number of comments

n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
n_comments = int(n_comments.text)


Report: PSG turns down Miralem Pjanic-Marco Verratti swap deal
Danny Penza
Apr 3, 2020, 3:07am PDT


In [57]:
# Looping through all comments to gather comment title, comment body, poster, and number of recs


comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
    comment_title = driver.find_element_by_xpath(xpath + '/div[1]')
    comment_body = driver.find_element_by_xpath(xpath + '/div[2]')
    
    # Generally, comment meta data is stored in the 3rd div except when commentors have a signature
    # in which case the meta data is in the 4th div
    
    if driver.find_elements_by_xpath(xpath + '/div[4]'):
        poster = driver.find_element_by_xpath(xpath + '/div[4]/span[1]/a[1]')
        post_date = driver.find_element_by_xpath(xpath + '/div[4]/span[1]/a[2]')
    
    # Some comments have no recommendations
        if driver.find_elements_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]'):
            recs = driver.find_element_by_xpath(xpath + '/div[4]/span[2]/button[2]/span[2]')
            recs = int(recs.text.replace('(','').replace(')',''))
        else:
            recs = 0
    else:
        poster = driver.find_element_by_xpath(xpath + '/div[3]/span[1]/a[1]')
        post_date = driver.find_element_by_xpath(xpath + '/div[3]/span[1]/a[2]')
        
    # Some comments have no recommendations
        if driver.find_elements_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]'):
            recs = driver.find_element_by_xpath(xpath + '/div[3]/span[2]/button[2]/span[2]')
            recs = int(recs.text.replace('(','').replace(')',''))
        else:
            recs = 0

    comment = pd.DataFrame([comment_title.text,comment_body.text,poster.text,post_date.text.split('|')[0],
                            post_date.text.split('|')[1],recs])
    comments_df = pd.concat([comments_df,comment.T])
    
    ;
   

#### Aggregating into data frame, exporting

In [58]:
comments_df.columns = ['Comment Title','Comment Body','Poster','Date','Time','Recs']

# Adding article title and author to comments df
comments_df.insert(0, 'Article', blog_title.text)
comments_df.insert(1, 'Author', article_author.text)
comments_df.insert(2, 'Publish Date', article_publish_date.text)


comments_df.head()



,Article,Author,Publish Date,Comment Title,Comment Body,Poster,Date,Time,Recs
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,"Apr 3, 2020, 3:07am PDT",,interesting video about how some stats should/...,resko,"Apr 3, 2020",6:37 AM,0
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,"Apr 3, 2020, 3:07am PDT",,Well that was a tough decision for PSG!,Paolo C,"Apr 3, 2020",6:42 AM,8
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,"Apr 3, 2020, 3:07am PDT",,hmmm…………maybe if we added Khedira……….or Bernar...,Ricardo 1,"Apr 3, 2020",10:44 AM,0
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,"Apr 3, 2020, 3:07am PDT",,Interest chart here. Various strikers over/und...,Paolo C,"Apr 3, 2020",9:59 AM,6
0,Report: PSG turns down Miralem Pjanic-Marco Ve...,Danny Penza,"Apr 3, 2020, 3:07am PDT",,Dybala is an amazing finisher\nso much this,#finoallafine,"Apr 5, 2020",11:50 PM,0


In [59]:
comments_df.to_csv('comments.csv', index=False)


In [40]:
driver.quit()